In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from PIL import Image
import pandas as pd

class VehicleDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    return running_loss / len(train_loader)

def evaluate(model, val_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    return accuracy, macro_f1

In [2]:
def main():
    # 设置随机种子
    torch.manual_seed(42)
    np.random.seed(42)
    
    # 设备配置
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"使用设备: {device}")
    
    # 数据预处理
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])
    
    # 加载数据
    data_dir = "Data/Data"  # 修正数据路径
    image_paths = []
    labels = []
    
    # 遍历文件夹收集图片路径和标签
    for filename in os.listdir(data_dir):
        if filename.endswith('.png'):
            # 从文件名中提取类别标签（1_xxx.png -> 0, 2_xxx.png -> 1, 3_xxx.png -> 2）
            class_id = int(filename.split('_')[0]) - 1  # 将1,2,3映射到0,1,2
            image_paths.append(os.path.join(data_dir, filename))
            labels.append(class_id)
    
    # 转换为numpy数组
    image_paths = np.array(image_paths)
    labels = np.array(labels)
    
    print(f"总样本数: {len(labels)}")
    print("类别分布:")
    for i in range(3):
        print(f"类别 {i+1}: {sum(labels == i)} 个样本")
    
    # 5折交叉验证
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    accuracies = []
    f1_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(image_paths, labels)):
        print(f"\nFold {fold + 1}")
        
        # 准备数据集
        train_dataset = VehicleDataset(
            [image_paths[i] for i in train_idx],
            [labels[i] for i in train_idx],
            transform=transform
        )
        val_dataset = VehicleDataset(
            [image_paths[i] for i in val_idx],
            [labels[i] for i in val_idx],
            transform=transform
        )
        
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=32)
        
        # 模型初始化
        model = models.resnet18(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, 3)  # 3个类别
        model = model.to(device)
        
        # 损失函数和优化器
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        
        best_score = 0
        best_epoch = 0
        patience = 3
        no_improve = 0
        
        # 训练循环
        for epoch in range(30):  # 增加训练轮数到30
            train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
            accuracy, macro_f1 = evaluate(model, val_loader, device)
            
            # 计算当前fold的得分
            current_score = 0.7 * accuracy + 0.3 * macro_f1
            
            print(f"Epoch {epoch+1}, Loss: {train_loss:.4f}, Accuracy: {accuracy:.4f}, Macro-F1: {macro_f1:.4f}, Score: {current_score:.4f}")
            
            # 早停策略
            if current_score > best_score:
                best_score = current_score
                best_epoch = epoch
                no_improve = 0
            else:
                no_improve += 1
                if no_improve >= patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break
        
        # 记录最终结果
        final_accuracy, final_macro_f1 = evaluate(model, val_loader, device)
        accuracies.append(final_accuracy)
        f1_scores.append(final_macro_f1)
        
        print(f"\nFold {fold + 1} Final Results:")
        print(f"Accuracy: {final_accuracy:.4f}")
        print(f"Macro-F1: {final_macro_f1:.4f}")
        fold_score = 0.7 * final_accuracy + 0.3 * final_macro_f1
        print(f"Fold Score: {fold_score:.4f}")
    
    # 计算5折交叉验证的平均结果
    mean_accuracy = np.mean(accuracies)
    std_accuracy = np.std(accuracies)
    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)
    
    print("\n最终五折交叉验证结果:")
    print(f"平均准确率: {mean_accuracy:.4f} ± {std_accuracy:.4f}")
    print(f"平均Macro-F1: {mean_f1:.4f} ± {std_f1:.4f}")
    
    # 计算最终平均得分
    final_score = 0.7 * mean_accuracy + 0.3 * mean_f1
    print(f"\n最终平均得分: {final_score:.4f}")

if __name__ == "__main__":
    main()

使用设备: cuda
总样本数: 1572
类别分布:
类别 1: 720 个样本
类别 2: 608 个样本
类别 3: 244 个样本

Fold 1


c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.2502, Accuracy: 0.6952, Macro-F1: 0.6377, Score: 0.6780
Epoch 2, Loss: 0.1502, Accuracy: 0.9492, Macro-F1: 0.9383, Score: 0.9459
Epoch 3, Loss: 0.1117, Accuracy: 0.9556, Macro-F1: 0.9512, Score: 0.9543
Epoch 4, Loss: 0.1432, Accuracy: 0.9460, Macro-F1: 0.9423, Score: 0.9449
Epoch 5, Loss: 0.1184, Accuracy: 0.9365, Macro-F1: 0.9253, Score: 0.9331
Epoch 6, Loss: 0.0658, Accuracy: 0.9619, Macro-F1: 0.9545, Score: 0.9597
Epoch 7, Loss: 0.0706, Accuracy: 0.9778, Macro-F1: 0.9710, Score: 0.9757
Epoch 8, Loss: 0.0611, Accuracy: 0.9365, Macro-F1: 0.9314, Score: 0.9350
Epoch 9, Loss: 0.0657, Accuracy: 0.9746, Macro-F1: 0.9697, Score: 0.9731
Epoch 10, Loss: 0.0369, Accuracy: 0.9619, Macro-F1: 0.9615, Score: 0.9618
Early stopping at epoch 10

Fold 1 Final Results:
Accuracy: 0.9619
Macro-F1: 0.9594
Fold Score: 0.9612

Fold 2


c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.2685, Accuracy: 0.6508, Macro-F1: 0.6532, Score: 0.6515
Epoch 2, Loss: 0.1397, Accuracy: 0.8762, Macro-F1: 0.8410, Score: 0.8656
Epoch 3, Loss: 0.1728, Accuracy: 0.8825, Macro-F1: 0.8818, Score: 0.8823
Epoch 4, Loss: 0.1353, Accuracy: 0.9619, Macro-F1: 0.9462, Score: 0.9572
Epoch 5, Loss: 0.1074, Accuracy: 0.9111, Macro-F1: 0.8933, Score: 0.9058
Epoch 6, Loss: 0.0715, Accuracy: 0.9683, Macro-F1: 0.9666, Score: 0.9678
Epoch 7, Loss: 0.0387, Accuracy: 0.9841, Macro-F1: 0.9810, Score: 0.9832
Epoch 8, Loss: 0.0499, Accuracy: 0.8889, Macro-F1: 0.8923, Score: 0.8899
Epoch 9, Loss: 0.0606, Accuracy: 0.9492, Macro-F1: 0.9335, Score: 0.9445
Epoch 10, Loss: 0.1048, Accuracy: 0.9683, Macro-F1: 0.9603, Score: 0.9659
Early stopping at epoch 10

Fold 2 Final Results:
Accuracy: 0.9619
Macro-F1: 0.9575
Fold Score: 0.9606

Fold 3


...

Epoch 1, Loss: 0.3167, Accuracy: 0.7548, Macro-F1: 0.7291, Score: 0.7471
Epoch 2, Loss: 0.1682, Accuracy: 0.9204, Macro-F1: 0.9081, Score: 0.9167
Epoch 3, Loss: 0.0870, Accuracy: 0.9745, Macro-F1: 0.9714, Score: 0.9736
Epoch 4, Loss: 0.0851, Accuracy: 0.9236, Macro-F1: 0.9125, Score: 0.9202
Epoch 5, Loss: 0.0669, Accuracy: 0.9140, Macro-F1: 0.8951, Score: 0.9083
Epoch 6, Loss: 0.0705, Accuracy: 0.9682, Macro-F1: 0.9623, Score: 0.9664
Early stopping at epoch 6

Fold 3 Final Results:
Accuracy: 0.9650
Macro-F1: 0.9580
Fold Score: 0.9629

Fold 4


c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.2674, Accuracy: 0.8694, Macro-F1: 0.8374, Score: 0.8598
Epoch 2, Loss: 0.1357, Accuracy: 0.9108, Macro-F1: 0.8966, Score: 0.9065
Epoch 3, Loss: 0.1222, Accuracy: 0.7803, Macro-F1: 0.7630, Score: 0.7751
Epoch 4, Loss: 0.1028, Accuracy: 0.9268, Macro-F1: 0.9133, Score: 0.9227
Epoch 5, Loss: 0.1056, Accuracy: 0.9682, Macro-F1: 0.9622, Score: 0.9664
Epoch 6, Loss: 0.0757, Accuracy: 0.9427, Macro-F1: 0.9367, Score: 0.9409
Epoch 7, Loss: 0.1720, Accuracy: 0.8631, Macro-F1: 0.8548, Score: 0.8606
Epoch 8, Loss: 0.1163, Accuracy: 0.9490, Macro-F1: 0.9395, Score: 0.9462
Early stopping at epoch 8

Fold 4 Final Results:
Accuracy: 0.9363
Macro-F1: 0.9209
Fold Score: 0.9317

Fold 5


c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\18794\Desktop\比赛\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.2914, Accuracy: 0.9140, Macro-F1: 0.9052, Score: 0.9114
Epoch 2, Loss: 0.1014, Accuracy: 0.8822, Macro-F1: 0.8700, Score: 0.8785
Epoch 3, Loss: 0.1056, Accuracy: 0.8726, Macro-F1: 0.8833, Score: 0.8758
Epoch 4, Loss: 0.0663, Accuracy: 0.9777, Macro-F1: 0.9783, Score: 0.9779
Epoch 5, Loss: 0.0844, Accuracy: 0.9076, Macro-F1: 0.9082, Score: 0.9078
Epoch 6, Loss: 0.1876, Accuracy: 0.7452, Macro-F1: 0.7264, Score: 0.7396
Epoch 7, Loss: 0.0950, Accuracy: 0.9713, Macro-F1: 0.9659, Score: 0.9697
Early stopping at epoch 7

Fold 5 Final Results:
Accuracy: 0.9777
Macro-F1: 0.9735
Fold Score: 0.9764

最终五折交叉验证结果:
平均准确率: 0.9606 ± 0.0135
平均Macro-F1: 0.9538 ± 0.0175

最终平均得分: 0.9585
